<a href="https://colab.research.google.com/github/rafaelventura204/3D_Segmentation/blob/main/3D_Segmentation_Active_Contour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, Accuracy


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, accuracy_score, jaccard_score

from skimage.segmentation import active_contour

from skimage import draw

Definizione immagine Sfera prolata

In [62]:
# Definizione della classe per creare una sfera prolata
class ProlateSpheroid:
    def __init__(self, size=128, a=48, c=64, shell_thickness=2):
        self.size = size
        self.equatorial_radius = a
        self.polar_radius = c
        self.shell_thickness = shell_thickness
        self.matrix = self.create_spheroid()

    def create_spheroid(self):
        volume = np.full((self.size, self.size, self.size), 255)  # Initialize all voxels to 255 (outside)
        center = np.array([self.size // 2, self.size // 2, self.size // 2])

        for x in range(self.size):
            for y in range(self.size):
                for z in range(self.size):
                    distance_from_center = ((x - center[0])**2 + (y - center[1])**2) / self.equatorial_radius**2 \
                                           + (z - center[2])**2 / self.polar_radius**2
                    # Inside the spheroid
                    if distance_from_center < 1:
                        volume[x, y, z] = 127  # Set voxels inside the spheroid to 127 (inside)
                    # Within the shell thickness
                    elif 1 <= distance_from_center < 1 + (self.shell_thickness/self.size)**2:
                        volume[x, y, z] = 0  # Set voxels within the shell thickness to 0 (shell)
        return volume

# Create a prolate spheroid with a thicker shell
spheroid = ProlateSpheroid(shell_thickness=10)

# Downsampling per la visualizzazione
downsampling_factor = 4
downsampled_matrix = spheroid.matrix[::downsampling_factor, ::downsampling_factor, ::downsampling_factor]

# Estrazione delle coordinate
x, y, z = np.where(downsampled_matrix != 255) #se commentata fa vedere la parte esterna

# Funzione per mappare i valori ai colori
def color_mapper(value):
    if value == 0:
        return 'black'  # shell
    elif value == 127:
        return 'blue'   # inside spheroid
    else:
        return 'yellow'  # outside spheroid

# Applicazione della funzione di mapping
colors = np.vectorize(color_mapper)(downsampled_matrix[x, y, z])

# Creazione del grafico 3D
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color=colors,  # colori dei punti
        opacity=0.8
    )
)])

# Aggiornamento del layout per una migliore visualizzazione
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X-axis'),
        yaxis=dict(title='Y-axis'),
        zaxis=dict(title='Z-axis (Polar Axis)'),
        aspectratio=dict(x=1, y=1, z=1.5)  # Rapporto d'aspetto per enfatizzare la forma prolata
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Visualizzazione della figura
fig.show()


Sotto matrice della matrice originale!

In [63]:
# Calcolo le coordinate centrali della matrice 3D
center = spheroid.size // 2
half_subportion = 32 // 2  # Metà della dimensione della sotto-porzione

# Prendo la sotto-porzione centrale della matrice 3D
subportion_matrix = spheroid.matrix[center-half_subportion:center+half_subportion,
                                    center-half_subportion:center+half_subportion,
                                    center-half_subportion:center+half_subportion]

# Verifico la dimensione della sotto-porzione per assicurarmi che sia corretta
subportion_matrix.shape

(32, 32, 32)

Estrazione di slice con spaziatura di 2mm dalla sotto matrice

In [64]:
# Correggiamo la funzione per estrarre le slice con una spaziatura di 2mm (equivalenti a 2 voxel se 1 voxel = 1 mm)
def extract_spaced_slices_correctly(matrix, spacing_mm, mm_per_voxel=1):
    """
    Estrae le slice distanziate da una specifica quantità in millimetri.

    :param matrix: Matrice 3D dalla quale estrarre le slice.
    :param spacing_mm: Spaziatura in millimetri tra le slice.
    :param mm_per_voxel: Quanti millimetri rappresenta ogni voxel (assumendo voxel cubici).
    :return: Un dizionario contenente le slice estratte lungo ogni asse.
    """
    # Calcoliamo quanti voxel corrispondono alla spaziatura in millimetri
    # Usiamo divisione normale per calcolare il numero di voxel per la spaziatura data
    spacing_voxels = spacing_mm / mm_per_voxel

    # Calcoliamo l'indice per iniziare a estrarre le slice
    start_index = spacing_voxels

    # Determiniamo gli indici per estrarre le slice con la spaziatura data
    # Si parte dall'indice start_index e si procede con passi di spacing_voxels + 1 per includere ogni spacing_voxels-esimo voxel
    indices = np.arange(start_index, matrix.shape[0], spacing_voxels).astype(int)

    # Estraiamo le slice distanziate
    slices_x = matrix[indices, :, :]
    slices_y = matrix[:, indices, :]
    slices_z = matrix[:, :, indices]

    return {
        'x': slices_x,
        'y': slices_y,
        'z': slices_z
    }

# Applichiamo la funzione corretta alla sotto-porzione centrata
spaced_slices_correct = extract_spaced_slices_correctly(subportion_matrix, spacing_mm=2, mm_per_voxel=1)

# Verifichiamo la dimensione delle slice estratte
slices_shapes_correct = {axis: spaced_slices_correct[axis].shape for axis in 'xyz'}
slices_shapes_correct

{'x': (15, 32, 32), 'y': (32, 15, 32), 'z': (32, 32, 15)}

PARTE PRIMA: Segmentazione 2D U-net

In [65]:
def generate_ground_truth_masks(slices_dict):
    masks_dict = {}
    for axis, slices in slices_dict.items():
        masks = (slices == 127).astype(np.float32)  # 1 per i pixel interni, 0 per il guscio e l'esterno
        masks_dict[axis] = masks
    return masks_dict

# Generiamo le maschere di ground truth
ground_truth_masks = generate_ground_truth_masks(spaced_slices_correct)

In [66]:
def unet(input_size=(32, 32, 1)):
    inputs = Input(input_size)

    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)

    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=3)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=3)
    conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=3)
    conv7 = Conv2D(16, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv7)

    conv8 = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=conv8)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=[Precision(name='precision')])


    return model

# Creazione del modello U-Net
model = unet()

Preparazione dati allenamento e addestramento modello

Pre-elaborazione delle Slice e delle Maschere di Ground Truth:

In [67]:
# Assumiamo che spaced_slices_correct sia il tuo dizionario con le slice e ground_truth_masks le tue maschere

# Preparazione delle immagini di input e delle maschere di ground truth
# Convertiamo i dizionari in liste e quindi in array numpy
X = np.array(spaced_slices_correct['x'])  # Utilizza 'y' o 'z' se preferisci quelle slice
y = np.array(ground_truth_masks['x'])  # Lo stesso vale per 'y' o 'z'

# Normalizzazione dei dati di input
X_norm = X / 255.0
y_norm = y  # Le maschere sono già 0 o 1, quindi non necessitano di normalizzazione

# Ridimensionamento aggiungendo una dimensione canale per Keras
X_norm = np.expand_dims(X_norm, axis=-1)
y_norm = np.expand_dims(y_norm, axis=-1)

# Divisione in set di allenamento e validazione
X_train, X_val, y_train, y_val = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

Configurazione dei Callback:



In [68]:
# Configurazione dei callback per salvare il miglior modello e fermare l'addestramento se non ci sono miglioramenti
checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

Addestramento del Modello:



In [69]:
# Costruzione del modello U-Net (utilizza la funzione unet fornita in precedenza)
model = unet(input_size=(32, 32, 1))

# Addestramento del modello
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.6661 - precision: 1.0000 - val_loss: 0.6634 - val_precision: 1.0000
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 0.6634 - precision: 1.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



1/1 [==============================] - 0s 165ms/step - loss: 0.6634 - precision: 1.0000 - val_loss: 0.6610 - val_precision: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 144ms/step - loss: 0.6610 - precision: 1.0000 - val_loss: 0.6585 - val_precision: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 147ms/step - loss: 0.6585 - precision: 1.0000 - val_loss: 0.6559 - val_precision: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 161ms/step - loss: 0.6559 - precision: 1.0000 - val_loss: 0.6532 - val_precision: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 159ms/step - loss: 0.6532 - precision: 1.0000 - val_loss: 0.6507 - val_precision: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 154ms/step - loss: 0.6507 - precision: 1.0000 - val_loss: 0.6481 - val_precision: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 157ms/step - loss: 0.6481 - precision: 1.0000 - val_loss: 0.6452 - val_precision: 1.0000
Epoch 9/50
1/1 [=

Valutazione del Modello:

In [70]:
# Valutazione del modello sui dati di validazione
model.evaluate(X_val, y_val)


1/1 [==============================] - 0s 23ms/step - loss: 8.1791e-04 - precision: 1.0000


[0.0008179086144082248, 1.0]

In [71]:
# Valutazione del modello sui dati di validazione per ottenere la precisione
val_loss, val_precision = model.evaluate(X_val, y_val)
print(f"Validation precision: {val_precision}")


1/1 [==============================] - 0s 23ms/step - loss: 8.1791e-04 - precision: 1.0000
Validation precision: 1.0


Visualizzazione delle Curve di Addestramento:



In [72]:
# @title
# Plot delle curve di addestramento
"""
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Plot delle curve di addestramento per la precisione
plt.plot(history.history['precision'], label='Training Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title('Precision Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.show()
"""

"\nplt.plot(history.history['loss'], label='Training Loss')\nplt.plot(history.history['val_loss'], label='Validation Loss')\nplt.legend()\nplt.title('Loss Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('Loss')\nplt.show()\n\n# Plot delle curve di addestramento per la precisione\nplt.plot(history.history['precision'], label='Training Precision')\nplt.plot(history.history['val_precision'], label='Validation Precision')\nplt.legend()\nplt.title('Precision Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('Precision')\nplt.show()\n"

In [73]:

# Calcola le predizioni sul set di validazione
y_pred = model.predict(X_val)
y_pred_thresholded = y_pred > 0.5

# Calcolare le metriche manualmente o utilizzare quelle di Keras
precision = Precision()
recall = Recall()
accuracy = Accuracy()

precision.update_state(y_val, y_pred_thresholded)
recall.update_state(y_val, y_pred_thresholded)
accuracy.update_state(y_val, y_pred_thresholded)

print(f'Precision: {precision.result().numpy()}')
print(f'Recall: {recall.result().numpy()}')
print(f'Accuracy: {accuracy.result().numpy()}')

# Calcolo dell'F1-score
f1_score = 2 * (precision.result().numpy() * recall.result().numpy()) / (precision.result().numpy() + recall.result().numpy() + 1e-7)
print(f'F1-score: {f1_score}')

# Calcolo dell'Intersection over Union (IoU)
# Calcola le predizioni binarie sogliando il risultato
y_pred_binary = (y_pred > 0.5).astype(int)

# Calcola le maschere binarie di ground truth
y_val_binary = (y_val > 0.5).astype(int)

# Calcolo dell'Intersection over Union (IoU)
intersection = np.logical_and(y_val_binary, y_pred_binary).sum()
union = np.logical_or(y_val_binary, y_pred_binary).sum()
iou = intersection / (union + 1e-7)

print(f'IoU (Intersection over Union): {iou}')



1/1 [==============================] - 0s 131ms/step
Precision: 1.0
Recall: 1.0
Accuracy: 1.0
F1-score: 0.9999999500000026
IoU (Intersection over Union): 0.9999999999674479


PARTE SECONDA: Segmentazione 3D Active Contour

In [80]:
alpha_values = [0.01, 0.015, 0.02]
beta_values = [8, 10, 12]
gamma_values = [0.001, 0.01, 0.1]

def segmenta_slice(spaced_slices, r, alpha, beta, gamma):
    segmented_slices = []
    for img in spaced_slices:
        # Definisci il centro della slice
        center_x = img.shape[1] // 2
        center_y = img.shape[0] // 2

        # Calcola le coordinate dell'ellisse
        s = np.linspace(0, 2*np.pi, 400)
        x = center_x + r * np.cos(s)
        y = center_y + r * np.sin(s)
        init = np.array([x, y]).T

        # Applicazione dell'algoritmo Active Contour
        snake = active_contour(img, init, alpha=alpha, beta=beta, gamma=gamma)
        segmented_slices.append(snake)
    return segmented_slices

r = min(spheroid.equatorial_radius, spheroid.polar_radius)
alpha, beta, gamma = 0.01, 10, 0.001  # Sostituisci con i tuoi valori ottimizzati
segmentazione_x = segmenta_slice(spaced_slices_correct['x'], r, alpha, beta, gamma)
segmentazione_y = segmenta_slice(spaced_slices_correct['y'], r, alpha, beta, gamma)
segmentazione_z = segmenta_slice(spaced_slices_correct['z'], r, alpha, beta, gamma)



"""
def segmenta_slice(spaced_slices, a, c, alpha=0.015, beta=10, gamma=0.001):
    segmented_slices = []
    center = [spaced_slices[0].shape[1] // 2, spaced_slices[0].shape[0] // 2]

    for i, img in enumerate(spaced_slices):
        # Utilizziamo una stima della sfera prolata per l'inizializzazione
        s = np.linspace(0, 2*np.pi, 400)
        x = center[0] + a * np.cos(s)
        y = center[1] + c * np.sin(s)
        init = np.array([x, y]).T

        # Applicazione dell'algoritmo Active Contour
        snake = active_contour(img, init, alpha=alpha, beta=beta, gamma=gamma)
        segmented_slices.append(snake)
    return segmented_slices


# Aggiorna i parametri 'a' e 'c' in base alle dimensioni della tua sfera prolata
segmentazione_x = segmenta_slice(spaced_slices_correct['x'], a=48, c=64)
segmentazione_y = segmenta_slice(spaced_slices_correct['y'], a=48, c=64)
segmentazione_z = segmenta_slice(spaced_slices_correct['z'], a=48, c=64)
# Ripeti per gli altri assi
"""

"\ndef segmenta_slice(spaced_slices, a, c, alpha=0.015, beta=10, gamma=0.001):\n    segmented_slices = []\n    center = [spaced_slices[0].shape[1] // 2, spaced_slices[0].shape[0] // 2]\n\n    for i, img in enumerate(spaced_slices):\n        # Utilizziamo una stima della sfera prolata per l'inizializzazione\n        s = np.linspace(0, 2*np.pi, 400)\n        x = center[0] + a * np.cos(s)\n        y = center[1] + c * np.sin(s)\n        init = np.array([x, y]).T\n\n        # Applicazione dell'algoritmo Active Contour\n        snake = active_contour(img, init, alpha=alpha, beta=beta, gamma=gamma)\n        segmented_slices.append(snake)\n    return segmented_slices\n\n\n# Aggiorna i parametri 'a' e 'c' in base alle dimensioni della tua sfera prolata\nsegmentazione_x = segmenta_slice(spaced_slices_correct['x'], a=48, c=64)\nsegmentazione_y = segmenta_slice(spaced_slices_correct['y'], a=48, c=64)\nsegmentazione_z = segmenta_slice(spaced_slices_correct['z'], a=48, c=64)\n# Ripeti per gli altri 

Valutazione

In [81]:
from sklearn.metrics import f1_score


def create_mask_from_snake(shape, snake):
    """Crea una maschera binaria da un contorno snake."""
    mask = np.zeros(shape, dtype=np.uint8)
    rr, cc = draw.polygon(snake[:, 1], snake[:, 0], shape)
    mask[rr, cc] = 1
    return mask

def calculate_metrics(gt_mask, pred_mask):
    """Calcola le metriche di valutazione."""
    accuracy = accuracy_score(gt_mask.flatten(), pred_mask.flatten())
    precision = precision_score(gt_mask.flatten(), pred_mask.flatten(), zero_division=0)
    recall = recall_score(gt_mask.flatten(), pred_mask.flatten())
    f1 = f1_score(gt_mask.flatten(), pred_mask.flatten())
    iou = np.sum(gt_mask & pred_mask) / np.sum(gt_mask | pred_mask)
    return accuracy, precision, recall, f1, iou

def evaluate_segmentation(segmentazioni, spaced_slices, subportion_matrix, axis):
    """Valuta la segmentazione per un asse specifico."""
    metrics = []
    for i, snake in enumerate(segmentazioni):
        # Calcola gli indici corrispondenti nella subportion_matrix
        index = min(i * 2, subportion_matrix.shape[0] - 1)  # Assicurati che l'indice non superi le dimensioni della matrice

        if axis == 'x':
            pred_mask = create_mask_from_snake(spaced_slices['x'][i].shape, snake)
            gt_mask = subportion_matrix[index, :, :] == 127
        elif axis == 'y':
            pred_mask = create_mask_from_snake(spaced_slices['y'][i].shape, snake)
            gt_mask = subportion_matrix[:, index, :] == 127
        else:  # axis == 'z'
            pred_mask = create_mask_from_snake(spaced_slices['z'][i].shape, snake)
            gt_mask = subportion_matrix[:, :, index] == 127

        # Assicurati che le dimensioni delle maschere siano coerenti
        min_shape = np.minimum(pred_mask.shape, gt_mask.shape)
        pred_mask_resized = pred_mask[:min_shape[0], :min_shape[1]]
        gt_mask_resized = gt_mask[:min_shape[0], :min_shape[1]]

        # Usa le maschere ridimensionate per il calcolo delle metriche
        metrics.append(calculate_metrics(gt_mask_resized, pred_mask_resized))
    return np.mean(metrics, axis=0)



# Valutazione per ciascun asse
metrics_x = evaluate_segmentation(segmentazione_x, spaced_slices_correct, subportion_matrix, 'x')
metrics_y = evaluate_segmentation(segmentazione_y, spaced_slices_correct, subportion_matrix, 'y')
metrics_z = evaluate_segmentation(segmentazione_z, spaced_slices_correct, subportion_matrix, 'z')

# Calcolo delle metriche medie complessive
mean_metrics = np.mean([metrics_x, metrics_y, metrics_z], axis=0)
print(f"Media Complessiva Accuracy: {mean_metrics[0]}")
print(f"Media Complessiva Precision: {mean_metrics[1]}")
print(f"Media Complessiva Recall: {mean_metrics[2]}")
print(f"Media Complessiva F1-Score: {mean_metrics[3]}")
print(f"Media Complessiva IoU: {mean_metrics[4]}")



Media Complessiva Accuracy: 1.0
Media Complessiva Precision: 1.0
Media Complessiva Recall: 1.0
Media Complessiva F1-Score: 1.0
Media Complessiva IoU: 1.0
